In [1]:
import torch

In [2]:
import numpy as np

In [3]:
inputs = np.array([
    [73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]], dtype='float32'
)
# inputs for temperature, rainfall and humidity respectively

In [4]:
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [6]:
targets = np.array([
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], dtype='float32'
)
# the targets represent the annual crop yields for apples and oranges respectively

In [10]:
# converting inputs and targets into tensors
# inputs = torch.from_numpy(inputs)
# targets = torch.from_numpy(targets)

# weights and biases
weight = torch.randn(2,3, requires_grad=True)
bias = torch.randn(2, requires_grad=True)


In [11]:
inputs @ weight.t() + bias
# the .t() means transpose the matrix

tensor([[ 52.7158,   9.9687],
        [ 86.0634,   6.6067],
        [ 68.4753,  51.9956],
        [ 33.9591,  -7.9663],
        [104.7419,  10.1228]], grad_fn=<AddBackward0>)

In [12]:
def model(x):
    return x @ weight.t() + bias

In [13]:
# generate predictions
predictions = model(inputs)
predictions

tensor([[ 52.7158,   9.9687],
        [ 86.0634,   6.6067],
        [ 68.4753,  51.9956],
        [ 33.9591,  -7.9663],
        [104.7419,  10.1228]], grad_fn=<AddBackward0>)

In [14]:
diff = predictions - targets
# mean squared error
torch.sum(diff * diff) / diff.numel()

tensor(3568.6992, grad_fn=<DivBackward0>)

In [16]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [17]:
loss = mse(predictions, targets)
# compute the gradients
loss.backward()

/home/gachuki/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:173: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484657607/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [18]:
print(weight)

tensor([[-0.3480, -0.1474,  2.0336],
        [-0.1018,  0.8566, -0.9295]], requires_grad=True)


In [19]:
print(weight.grad)

tensor([[ -566.9204, -1172.6605,  -436.6346],
        [-6423.7100, -7113.8125, -4521.1860]])


In [20]:
weight -= weight.grad * 1e-5
bias -= bias.grad * 1e-5

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [21]:
weight.grad.zero_()
bias.grad.zero_()

tensor([0., 0.])

In [29]:
# to train for 100 epochs
for i in range(100):
    predictions = model(inputs)
    loss = mse(predictions, targets)
    loss.backward()
    with torch.no_grad():
        weight -= weight.grad * 1e-5
        bias -= bias.grad * 1e-5
        weight.grad.zero_()
        bias.grad.zero_()

In [30]:
loss

tensor(76.3726, grad_fn=<DivBackward0>)

tensor(92.0277, grad_fn=<DivBackward0>)